### CW_model_training.ipynb 

This is based on the original notebook by the main author of the paper (`RADIPOP_model_training.ipynb`).
Since I need to reuse it on new data, I might as well clean it up a bit. 


However, currently it is not finished... #TODO 

In [2]:
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path
import yaml
import pandas as pd
import numpy as np
import pickle
import numba
from typing import Literal, Union
from glob import glob
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import roc_auc_score, roc_curve, r2_score, RocCurveDisplay
# see https://stackoverflow.com/questions/60321389/sklearn-importerror-cannot-import-name-plot-roc-curve

import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing  import StandardScaler


from sklearn.base import BaseEstimator, TransformerMixin
from scipy.cluster import hierarchy
from scipy.spatial.distance import squareform
from scipy.stats import spearmanr, pearsonr
from scipy.stats import ttest_ind
from collections import defaultdict
import seaborn as sns
import skopt
import time
import re 

import radipop_utils 
import radipop_utils.visualization
import radipop_utils.features
from radipop_utils.features import SpearmanReducerCont
import radipop_utils.utils
from radipop_utils.utils import get_files_dict_by_regex_pattern

import radipop_utils.data

from sklearn.metrics import mean_squared_error, mean_absolute_error




# load user/ system specific env variables:
from dotenv import dotenv_values, find_dotenv
config = dotenv_values(find_dotenv())  # load environment variables as dictionary

path = Path(os.path.abspath(radipop_utils.__file__))
RADIPOP_PACKAGE_ROOT = path.parent.parent



In [3]:

##------  You will likely need to change this 
DATA_ROOT_DIRECTORY = Path(config["DATA_ROOT_DIRECTORY"])
OUTDIR = DATA_ROOT_DIRECTORY / "radiomics" / "Dataset125_LSS" 
DATASET = "Dataset125_LSS"
RADIOMICS_OPTION = "radipop_111"
NUM_SEARCHES = 10
SEARCH_SCORING_METRIC = "r2"  # "neg_root_mean_squared_error"  "r2"
SAVE_RESULTS = True
##-----------

### Preparing the the data: 
- load radiomics and HVPG values 
- utilize our custom split (previously defined and stratified on sex, scanner, status)
- normalized the data

In [4]:
# load features and combine with predicted values: 

df_Tr, df_iTs, df_eTs = radipop_utils.data.load_HVPG_values_and_radiomics(DATASET=DATASET, RADIOMICS_OPTION=RADIOMICS_OPTION, DATA_ROOT_DIRECTORY=DATA_ROOT_DIRECTORY)
print(f"{len(df_Tr)=}, {len(df_eTs)=}, {len(df_iTs)=}")

len(df_Tr)=298, len(df_eTs)=50, len(df_iTs)=50


In [ ]:
# display(df_Tr)
# display(df_iTs)
# display(df_eTs)

In [1]:
split_indices_CV5_Tr = radipop_utils.data.extract_CV_indices(df_Tr)

X_Tr, Y_Tr, X_iTs, Y_iTs, X_eTs, Y_eTs = radipop_utils.data.preprocess_data(df_Tr, df_iTs, df_eTs, normalize_X=True)    

NameError: name 'radipop_utils' is not defined

In [ ]:
#plot dendrogram
corr = spearmanr(X_Tr).correlation

# Ensure the correlation matrix is symmetric
corr = (corr + corr.T) / 2
np.fill_diagonal(corr, 1)

# plt.matshow(corr)
# plt.show()

# We convert the correlation matrix to a distance matrix before performing
# hierarchical clustering using Ward's linkage.
distance_matrix = 1 - np.abs(corr)
dist_linkage = hierarchy.ward(squareform(distance_matrix))
plt.figure()
dendro = hierarchy.dendrogram(
    dist_linkage, no_labels=True
)

In [ ]:
#decide on a rought range for the cut parameters for dendrogram
split_params = [0.5, 0.75, 1, 2.75,  5, 7.5, 10]
for split_param in split_params:
    selector = SpearmanReducerCont(split_param=split_param)
    print(f"Selected features at height {split_param}:", len(selector.fit(X_Tr, Y_Tr).selected_features))
    
    

#### Fit on `Tr` data with CV and estimate best model + hyper parameters

In [ ]:
# Bounds for hyperparameters
param_bounds_rf = {
    'feature_selection__split_param' : skopt.space.Real(1,5, prior = "uniform"),
    'regression' : [RandomForestRegressor(random_state=2023)],
    'regression__n_estimators': skopt.space.Integer(100, 2000),
    'regression__max_depth': skopt.space.Integer(1, 50),
    'regression__min_samples_split': skopt.space.Integer(2, 25)#,
}


param_bounds_en = {
                 'feature_selection__split_param' : skopt.space.Real(1,5, prior = "uniform"),
                 'regression' : [ElasticNet(random_state=2023)],
                 'regression__alpha': skopt.space.Real(0.0001, 1.0, 'uniform'),
                 'regression__l1_ratio': skopt.space.Real(0, 1.0, 'uniform')
}


In [ ]:
#create a pipeline
reg_RF = Pipeline([
  #('scaler', StandardScaler()),  
  ('feature_selection', SpearmanReducerCont()),
  ('regression', RandomForestRegressor())
]) 

# cv5 = KFold(5, shuffle=True, random_state=2023)



In [ ]:


#try out models
opt0 = skopt.BayesSearchCV(
    reg_RF,
    [(param_bounds_en, NUM_SEARCHES), (param_bounds_rf, NUM_SEARCHES)],
    cv=split_indices_CV5_Tr,
    scoring=SEARCH_SCORING_METRIC,
    verbose=True,
    random_state=2023,
    n_jobs = 6
)
opt0.fit(X_Tr, Y_Tr)

display(opt0.best_params_)

In [ ]:
cv_res = pd.DataFrame(opt0.cv_results_)
cv_res
cv_res.iloc[:, :].reset_index().loc[:, "mean_test_score"].plot()
plt.grid()



In [ ]:
cv_res



In [ ]:
cv_res_sorted = cv_res.sort_values("rank_test_score")
pd.DataFrame(list(cv_res_sorted["params"]))
#cv_res_sorted[["params", "mean_test_score"]]



#### save model trained on the whole training data set and optimal paramters

In [ ]:


idx_best_EN_model = cv_res["mean_test_score"][:NUM_SEARCHES].argmax()
idx_best_RF_model = cv_res["mean_test_score"][NUM_SEARCHES:].argmax() + NUM_SEARCHES

if SAVE_RESULTS:
    os.makedirs(OUTDIR / "regression" / RADIOMICS_OPTION, exist_ok=True)
    dst = OUTDIR / "regression" / RADIOMICS_OPTION / f"Bayesian_results_{NUM_SEARCHES}_iterations_RFvsEN.xlsx"
    cv_res.to_excel(dst)
    print("Saved hyperparams search to : ", dst)

    
    save_dst = OUTDIR / "regression" / RADIOMICS_OPTION
    
    # save optimal parameters as yaml: 
    dst = save_dst / "SpearmanRed1_RF_opt_params.yml"
    data = {**cv_res.iloc[idx_best_RF_model, :].params}
    if "regression" in data: 
        data.pop("regression")
    with open(dst, 'w') as outfile:
        yaml.dump(data, outfile, default_flow_style=False)
        print("saved params to ", dst)
        

    # save optimal parameters as yaml: 
    dst = save_dst / "SpearmanRed1_EN_opt_params.yml"
    data = {**cv_res.iloc[idx_best_EN_model, :].params} # make copy to keep original dict unchanged
    if "regression" in data: 
        data.pop("regression")
    with open(dst, 'w') as outfile:
        yaml.dump(data, outfile, default_flow_style=False)
        print("saved params to ", dst)
        

    #---- set best performing en/rf models
    #create a pipeline
    reg_RF = Pipeline([
    #('scaler', StandardScaler()),  
    ('feature_selection', SpearmanReducerCont()),
    ('regression', RandomForestRegressor())
    ]) 
    #Set params
    np.random.seed(2023)
    reg_RF.set_params(**cv_res.iloc[idx_best_RF_model, :].params)
    reg_RF.fit(X_Tr, Y_Tr)
    dst = save_dst / f"SpearmanRed1_RF_opt.p"
    with open(dst, "wb") as fp:
        pickle.dump(reg_RF, fp)
        print("Saved model to ", dst)


    #create a pipeline
    reg_EN = Pipeline([
    #('scaler', StandardScaler()),  
    ('feature_selection', SpearmanReducerCont()),
    ('regression', ElasticNet())
    ]) 
    reg_EN.set_params(**cv_res.iloc[idx_best_EN_model, :].params)
    reg_EN.fit(X_Tr, Y_Tr)
    dst = save_dst / f"SpearmanRed1_EN_opt.p"
    with open(dst, "wb") as fp:
        pickle.dump(reg_EN, fp)
        print("Saved model to ", dst)
